In [27]:
import ktrain
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # use only CPU in prediction


In [28]:
predictor = ktrain.load_predictor('distilbert')

In [29]:
data = ['this movie is really great. deserve lots of praise']

In [30]:
predictor.predict(data)

['pos']

In [31]:
%%time
predictor.predict(data)

Wall time: 689 ms


['pos']

In [1]:
import requests
import json

In [ ]:
# This should be the code in app.py
# from flask import Flask, request, jsonify
# import distilbert_model as model


# app = Flask(__name__)


# @app.route('/')
# def hello():
#         return 'Congrats! Server is working'

# # get the json data
# @app.route('/get_sentiment', methods = ['POST'])
# def get_sentiment():
#         tx = request.get_json(force = True)
#         text = tx['Review']

#         sent = model.get_prediction(text)

#         print('get_sentiment  is working')

#         return jsonify(result = sent)

# if __name__ == '__main__':
#         app.run(host = '0.0.0.0', port = 5000, debug = True, use_reloader = False)

In [ ]:
# This code is to be written in distilbert_model.py
# import ktrain
# import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # use only CPU in prediction

# predictor = ktrain.load_predictor('distilbert')


# def get_prediction(x):
#         sent = predictor.predict([x])
#         return sent[0]

In [ ]:
# This code is written in bertmodel.py
# from flask import Flask, request, jsonify                                         
# import distilbert_model as model                                                  
                                                                                  
                                                                                  
# app = Flask(__name__)                                                             
                                                                                  
                                                                                  
# @app.route('/')                                                                   
# def hello():                                                                      
#         return 'Congrats! Server is working'                                      
                                                                                  
# # get the json data                                                               
# @app.route('/get_sentiment', methods = ['POST'])                                  
# def get_sentiment():                                                              
#         tx = request.get_json(force = True)                                       
#         text = tx['Review']                                                       
                                                                                  
#         sent = model.get_prediction(text)                                         
                                                                                  
#         print('get_sentiment  is working')                                        
                                                                                  
#         return jsonify(result = sent)                                             
                                                                                  
# if __name__ == '__main__':                                                        
#         app.run(host = '0.0.0.0', port = 5000, debug = True, use_reloader = False)

In [ ]:
# This code is to be written in wsgi.py
# from bertmodel import app
# if __name__ == "__main__":
#         app.run()

In [40]:
host = 'localhost'
port = 5000

url = "http://localhost:5000/"

x = requests.get(url)
print(x)
print(x.text)

<Response [200]>
Congrats! Server is working


In [41]:
data = {'Review': 'This movie is great. I have enjoyed a lot'}
data = json.dumps(data)



In [42]:
data

'{"Review": "This movie is great. I have enjoyed a lot"}'

In [43]:
url = "http://localhost:5000/get_sentiment"

x = requests.post(url, data)
print(x)
print(x.text)

<Response [200]>
{
  "result": "pos"
}



In [44]:
url = "http://localhost:5000/get_sentiment"
data = {'Review': 'this movie is horrible. return my money'}
data = json.dumps(data)

x = requests.post(url, data)
print(x.text)

{
  "result": "neg"
}



In [49]:
response = json.loads(x.text)

In [50]:
type(response)

dict

In [51]:
response.keys()

dict_keys(['result'])

In [52]:
response['result']

'neg'

## Predict from EC2 Instance -Windows server

In [53]:
url = "http://18.221.49.245:5000/get_sentiment"
data = {'Review': 'this movie is horrible. return my money'}
data = json.dumps(data)

x = requests.post(url, data)
print(x.text)

{
  "result": "neg"
}



## Predict from EC2 Instance - Ubuntu server

In [1]:
import requests
import json

In [3]:
url = "http://34.239.103.42:5000/get_sentiment"
data = {'Review': 'this movie is horrible. return my money'}
data = json.dumps(data)

x = requests.post(url, data)
print(x.text)

{
  "result": "neg"
}



In [4]:
%%timeit
url = "http://34.239.103.42:5000/get_sentiment"
data = {'Review': 'this movie is horrible. return my money'}
data = json.dumps(data)

x = requests.post(url, data)
print(x.text)

{
  "result": "neg"
}

{
  "result": "neg"
}

{
  "result": "neg"
}

{
  "result": "neg"
}

{
  "result": "neg"
}

{
  "result": "neg"
}

{
  "result": "neg"
}

{
  "result": "neg"
}

541 ms ± 5.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%time
url = "http://34.239.103.42:5000/get_sentiment"
data = {'Review': 'this movie is horrible. return my money'}
data = json.dumps(data)

x = requests.post(url, data)
print(x.text)
# since we are using free tier t2.micro instance with less memory the time taken for prediction is 
# more

{
  "result": "neg"
}

Wall time: 531 ms


## Predict from NGINX-EC2 instance 

In [1]:
import requests,json


In [7]:
url = "http://174.129.143.128"
x = requests.get(url, data)
print(x.text)

Congrats! Server is working


In [4]:
%%time
url = "http://174.129.143.128/get_sentiment"
data = {'Review': 'this movie is horrible. return my money'}
data = json.dumps(data)

x = requests.post(url, data)
print(x.text)

<html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.18.0 (Ubuntu)</center>
</body>
</html>

Wall time: 1min


The above error is due to multithreading in tensorflow
Make changes in bertmodel.ini

[uwsgi]

module = wsgi:app
    
master = true

processes = 5

socket = bertmodel.sock

chmod-socket = 660

vacuum = true

die-on-term = true 
 
modify the above code as follows:
    
module = wsgi:app
master = false
processes = 1
cheaper = 0
socket = bertmodel.sock
chmod-socket = 660
vacuum = true
die-on-term = true


In [8]:
%%time
url = "http://174.129.143.128/get_sentiment"
data = {'Review': 'this movie is horrible. return my money'}
data = json.dumps(data)

x = requests.post(url, data)
print(x.text)

{"result":"neg"}

Wall time: 19.5 s


In [9]:
%%timeit
data = {'Review': 'I like this movie'}
data = json.dumps(data)

x = requests.post(url, data)
print(x.text)

{"result":"pos"}

{"result":"pos"}

{"result":"pos"}

{"result":"pos"}

{"result":"pos"}

{"result":"pos"}

{"result":"pos"}

{"result":"pos"}

555 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
